In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests

In [3]:
options = webdriver.ChromeOptions()
options.binary_location = '/usr/bin/google-chrome-stable'
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('window-size=1200x600')

In [ ]:
driver = webdriver.Chrome(options=options)

In [5]:
driver.get('https://images.google.com')

In [6]:
# Wait for page to load
driver.implicitly_wait(5)

In [7]:
# Enter a search in the box, this case "cactus in desert"
searchbox = driver.find_element_by_css_selector('input[type=text]')
searchbox.send_keys('cactus in desert')
searchbox.send_keys(Keys.ENTER)

In [11]:
# Open the tool bar to advanced search
tools = driver.find_element_by_css_selector('#hdtb-tls')
tools.click()

In [15]:
# Use images that can be reused (optional)
usage_rights = driver.find_element_by_css_selector('#hdtbMenus > div > div:nth-child(8)')
usage_rights.click()
usage_rights_reuse = driver.find_element_by_css_selector('#sur_fmc')
usage_rights_reuse.click()

In [19]:
# Select only photos
imagetype = driver.find_element_by_css_selector('#hdtbMenus > div > div:nth-child(11)')
imagetype.click()
imagetype_photo = driver.find_element_by_css_selector('#itp_photo')
imagetype_photo.click()

In [23]:
# Avoid extra large/small images
imagesize = driver.find_element_by_css_selector('#hdtbMenus > div > div:nth-child(2)')
imagesize.click()
imagesize_medium = driver.find_element_by_css_selector('#isz_m')
imagesize_medium.click()

In [27]:
# Inspect the screenshot that search looks good
driver.save_screenshot('images_google.png')

True

In [26]:
# Scroll to the bottom, waiting to avoid bot detection
element = driver.find_element_by_tag_name("body")
for i in range(80):
    element.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)

In [28]:
# Switching from chrome to BeautifulSoup, chromedriver over longer periods will break
for_bs = driver.page_source

In [30]:
page_bs = BeautifulSoup(for_bs)

In [31]:
# Select each image box
images = page_bs.find_all("div", {"class": "rg_bx"})

In [32]:
len(images)

400

In [35]:
# Create a list of parsed json from each image box
image_list = []
for img in images:
    image_list.append(json.loads(img.find('div', {'class': 'rg_meta notranslate'}).contents[0]))

In [36]:
# Convert to a dataframe
image_df = pd.DataFrame(image_list)

In [41]:
# Adding a header for requests, some servers will reject requests that aren't from a browser
headers = {
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
}

In [50]:
# Download each image, has a timeout to avoid overloading the same server
for i in range(len(image_df.index)):
    time.sleep(2)
    img_url = image_df['ou'][i]
    img_ext = image_df['ity'][i]
    img_name = str(i) + '.' + img_ext
    response = requests.get(img_url, headers=headers)
    if response.ok:
        with open(img_name, 'w+') as img_file:
            img_file.write(response.content)
    else:
        print("Error downloading: "+ img_url)
    if i % 10 == 0:
        print("Images Downloaded: " + str(i))

Images Downloaded: 0
Images Downloaded: 10
Images Downloaded: 20
Images Downloaded: 30
Images Downloaded: 40
Images Downloaded: 50
Images Downloaded: 60
Images Downloaded: 70
Images Downloaded: 80
Images Downloaded: 90
Images Downloaded: 100
Images Downloaded: 110
Images Downloaded: 120
Images Downloaded: 130
Images Downloaded: 140
Images Downloaded: 150
Images Downloaded: 160
Images Downloaded: 170
Images Downloaded: 180
Images Downloaded: 190
Images Downloaded: 200
Images Downloaded: 210
Images Downloaded: 220
Images Downloaded: 230
Images Downloaded: 240
Images Downloaded: 250
Images Downloaded: 260
Images Downloaded: 270
Images Downloaded: 280
Images Downloaded: 290
Images Downloaded: 300
Images Downloaded: 310
Images Downloaded: 320
Images Downloaded: 330
Images Downloaded: 340
Images Downloaded: 350
Images Downloaded: 360
Images Downloaded: 370
Images Downloaded: 380
Images Downloaded: 390


In [ ]:
# Save images to a zip
zipf = zipfile.ZipFile('images2.zip', 'w', zipfile.ZIP_DEFLATED)

In [ ]:
for f in os.listdir('.'):
    if os.path.splitext(f)[1] in set(image_df['ity']):
        zipf.write(f)

In [ ]:
for f in os.listdir('.'):
    if os.path.splitext(f)[1] in set(image_df['ity']):
        os.remove(f)